<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Описание-данных" data-toc-modified-id="Описание-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Описание данных</a></span></li><li><span><a href="#Обработка-пропусков" data-toc-modified-id="Обработка-пропусков-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Обработка пропусков</a></span></li><li><span><a href="#Поиск-и-обработка-дубликатов" data-toc-modified-id="Поиск-и-обработка-дубликатов-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Поиск и обработка дубликатов</a></span></li><li><span><a href="#Проверка-данных-на-дисбаланс-классов" data-toc-modified-id="Проверка-данных-на-дисбаланс-классов-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Проверка данных на дисбаланс классов</a></span></li><li><span><a href="#Тестирование-метода-борьбы-с-дисбалансом-на-модели-логистической-регрессии" data-toc-modified-id="Тестирование-метода-борьбы-с-дисбалансом-на-модели-логистической-регрессии-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Тестирование метода борьбы с дисбалансом на модели логистической регрессии</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Определение-параметров-и-функций" data-toc-modified-id="Определение-параметров-и-функций-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Определение параметров и функций</a></span></li><li><span><a href="#Модель-LogisticRegression" data-toc-modified-id="Модель-LogisticRegression-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Модель LogisticRegression</a></span></li><li><span><a href="#Модель-RandomForest" data-toc-modified-id="Модель-RandomForest-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Модель RandomForest</a></span></li><li><span><a href="#Модель-CatBoost" data-toc-modified-id="Модель-CatBoost-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Модель CatBoost</a></span></li><li><span><a href="#Тестирование-лучшей-модели-с-векторизацией-данных-TF-IDF" data-toc-modified-id="Тестирование-лучшей-модели-с-векторизацией-данных-TF-IDF-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Тестирование лучшей модели с векторизацией данных TF-IDF</a></span></li><li><span><a href="#Модель-BERT" data-toc-modified-id="Модель-BERT-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Модель BERT</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других.

**Цель** — магазину «Викишоп» нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

**Задача исследования** — построить модель машинного обучения позволяющую классифицировать комментарии на позитивные и негативные.

**Исследование пройдёт в три этапа:**
 1. Загрузка и подготовка данных;
 2. Обучение и тестирование моделей;
 3. Выводы.
 
*Примечания:* 
* Для оценки качества моделей используется метрика F1. Значение метрики F1 на тестовой выборке должно быть не меньше 0.75.

## Подготовка

In [1]:
# импорт библиотек
import pandas as pd
import numpy as np
import torch
import transformers
import matplotlib.pyplot as plt
import re
import nltk
import spacy

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import Pool, CatBoostClassifier, cv

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve, precision_score, recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin 

from tqdm import notebook
from pymystem3 import Mystem
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
!spacy download en_core_web_sm

C:\Users\m1kha\anaconda3\envs\ds_practicum_env\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\m1kha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\m1kha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\m1kha\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\m1kha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


     ---------------------------------------- 12.8/12.8 MB 9.0 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


C:\Users\m1kha\anaconda3\envs\ds_practicum_env\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2022-09-29 10:22:11.706926: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-09-29 10:22:11.706952: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-29 10:22:11.709002: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: DESKTOP-9K9TKS6
2022-09-29 10:22:11.709048: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: DESKTOP-9K9TKS6


In [2]:
# сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', None)
 
# сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)
 
# сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)

Данные для исследования получаем из файла `toxic_comments`:

In [3]:
# чтение файлов с данными и сохранение в df
try:
    df = pd.read_csv('datasets/toxic_comments.csv', index_col=0)
except:
    df = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)

In [4]:
# получение первых пяти строк таблицы
display(df.head())

,text,toxic
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,"You, sir, are my hero. Any chance you remember what page that's on?",0


In [5]:
# получение общей информации о данных
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


### Описание данных

*	`text` — текст комментария;
*	`toxic` — целевой признак.

###  Обработка пропусков

In [6]:
# подсчёт доли пропусков 
display(df.isna().mean())

text     0.0
toxic    0.0
dtype: float64

В данных не содержатся пропуски.

### Поиск и обработка дубликатов

In [7]:
# подсчет явных дубликатов
print("Явных дубликатов в df:", df.duplicated().sum())

Явных дубликатов в df: 0


В данных не содержится дубликатов.

### Проверка данных на дисбаланс классов

Оценим баланс классов целевого признака в столбце `toxic`:

In [8]:
# подсчет количества значений целевого признака
df['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [9]:
# подсчет доли положительных/отрицательных значений
print('Доля положительные значений: {:.2%}'.format(df['toxic'].mean()))
print('Доля отрицательных значений: {:.2%}'.format(1-df['toxic'].mean()))

Доля положительные значений: 10.16%
Доля отрицательных значений: 89.84%


Присутствует явное смещение в сторону отрицательных значений.    

Метод увеличения выборки (upsampling) не имеет смысла использовать ввиду изначально большого количества данных. С другой стороны, так как выборка сильно несбалансирована, то применяя метод уменьшения выборки (downsampling) мы лишимся большого количества данных, важных для обучения, что скажется на итоговом качестве модели.    

Поэтому, принимаем решение в качестве метода борьбы с дисбалансом классов использовать взвешивание (balanced).

### Тестирование метода борьбы с дисбалансом на модели логистической регрессии

Протестируем выбранный метод борьбы с дисбалансом на модели логистической регрессии. В качестве определяющей метрики будем использовать F1-меру.

**Лемматизация данных, подготовка признаков**

In [10]:
#функция лемматизации и очистки символов
nlp = spacy.load("en_core_web_sm")
def lemmatize(text):
    text = text.lower()
    lemm_list = nlp(text) #лемматизация
    lemm_text = " ".join([token.lemma_ for token in lemm_list]) #объединяем элементы списка в строку, разделив их пробелом
    text_clear = re.sub(r'[^a-zA-Z]', ' ', lemm_text) #оставляем в тексте только латинские символы
    lemm_text_clear = " ".join(text_clear.split()) #объединяем элементы списка в строку, удалив лишние пробелом
    
    return lemm_text_clear

In [11]:
%%time
df['lemm_text'] = df['text'].apply(lambda x: lemmatize(x))

CPU times: total: 23min 14s
Wall time: 23min 15s


In [12]:
display("Исходный и лемматизированный текст:", df.head(1))

'Исходный и лемматизированный текст:'

,text,toxic,lemm_text
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,explanation why the edit make under my username hardcore metallica fan be revert they be not vandalism just closure on some gas after I vote at new york dolls fac and please do not remove the template from the talk page since I be retire now


In [13]:
#удаление исходного корпуса текстов
df = df.drop(['text'], axis=1)

In [14]:
#создание выборки с признаками и целевыми признаками
features = df.drop('toxic', axis=1)
target = df['toxic']

**Разделение данных на выборки**

Разделим данные на обучающую (80%) и тестовую (20%) выборки:

In [15]:
# разделение данных на обучающую и тестовую выборки
features_train, features_test = train_test_split(features.copy(), train_size=0.8,
                                                 test_size=0.2, random_state=12345)
target_train, target_test = train_test_split(target.copy(), train_size=0.8,
                                             test_size=0.2, random_state=12345)

In [16]:
print("Процент данных в обучающей выборке признаков:", '{:.0%}'.format(len(features_train)/len(features)))
print("Процент данных в тестовой выборке признаков", '{:.0%}'.format(len(features_test)/len(features)))
print()
print("Процент данных в обучающей выборке целевых признаков:", '{:.0%}'.format(len(target_train)/len(target)))
print("Процент данных в тестовой выборке целевых признаков", '{:.0%}'.format(len(target_test)/len(target)))

Процент данных в обучающей выборке признаков: 80%
Процент данных в тестовой выборке признаков 20%

Процент данных в обучающей выборке целевых признаков: 80%
Процент данных в тестовой выборке целевых признаков 20%


**Пайплайн для моделей и оценки TF-IDF**

In [17]:
#стоп-слова
stopwords = set(stopwords.words('english')) 

Создадим пайплайн, где соединим подсчет величины TF-IDF для корпуса текстов и модели обучения: 

In [18]:
#пайплан TF-IDF и модель логистической регрессии
model_logit_unbalanced = make_pipeline(CountVectorizer(stop_words=stopwords), TfidfTransformer(),
                            LogisticRegression(random_state=12345, max_iter=300))

model_logit_balanced = make_pipeline(CountVectorizer(stop_words=stopwords), TfidfTransformer(),
                            LogisticRegression(random_state=12345, max_iter=300, class_weight='balanced'))

**Обучение модели без баланса классов**

In [19]:
%%time
# обучение модели
f1_unbalanced = cross_val_score(model_logit_unbalanced, features_train['lemm_text'].values,
                                target_train, cv=5, scoring='f1', n_jobs=-1).mean()

# вывод результатов
print('F1:', round(f1_unbalanced, 3))
print()

F1: 0.732

CPU times: total: 828 ms
Wall time: 12 s


**Обучение модели с взвешенными классами**

In [20]:
%%time
# обучение модели
f1_balanced = cross_val_score(model_logit_balanced, features_train['lemm_text'].values,
                              target_train, cv=5, scoring='f1', n_jobs=-1).mean()

# вывод результатов
print('F1:', round(f1_balanced, 3))
print()

F1: 0.747

CPU times: total: 562 ms
Wall time: 12.6 s


**Итоги:**

In [21]:
# итоги обучения моделей
results_balancing = pd.DataFrame({'Unbalanced': f1_unbalanced, 'Balanced': f1_balanced},
                                 index=['F1'])
display(round(results_balancing, 3))

,Unbalanced,Balanced
F1,0.732,0.747


💡 Присутствует некоторое улучшение в целевой метрике, поэтому будем использовать взвешивание классов для баласировки выборки.

### Выводы

💡 В исходном файле содержится набор данных с комментариями пользователей сервиса размеченный по токсичности правок. 

В ходе первичного анализа и предобработки данных было выполнено:     
* В данных не выявлены пропуски или дубликаты;   

* Данные сильно не сбалансированы, поэтому будем применять "взвешивание классов" (class_weight='balanced');    

* Был выполнен расчет статистической меры TF-IDF, используемой для оценки важности слова в контексте корпуса текстов.

## Обучение

### Определение параметров и функций

In [22]:
parameters_logit = {'logisticregression__solver': ['lbfgs', 'newton-cg', 'sag', 'saga'],
                   'logisticregression__penalty': ['l2', 'none']}

parameters_RF = {'randomforestclassifier__max_depth': list(range(10, 221, 30))
                }
                 
parameters_CB = {'catboostclassifier__learning_rate': [0.1, 0.2]
                }

model_logit = make_pipeline(CountVectorizer(stop_words=stopwords), TfidfTransformer(),
                            LogisticRegression(random_state=12345, max_iter=2000, class_weight='balanced'))

model_RF =  make_pipeline(CountVectorizer(stop_words=stopwords), TfidfTransformer(),
                          RandomForestClassifier(random_state=12345, class_weight='balanced'))

model_CB = make_pipeline(CountVectorizer(stop_words=stopwords), TfidfTransformer(),
                         CatBoostClassifier(random_state=12345, verbose = False))

cv = 3 #количество итераций при кросс-валидации

In [23]:
def grid_search_cv(model, features_train, target_train, parameters):
       
    grid = GridSearchCV(estimator=model, param_grid=parameters, cv=cv, scoring='f1',
                        n_jobs=-1, error_score='raise')
    
    grid.fit(features_train, target_train)
    
    F1 = grid.best_score_
    best_params = grid.best_params_
    
    print(f'F1:', round(F1, 3))
    print("Параметры лучшей модели:", best_params)
    print()
    
    return F1

### Модель LogisticRegression

Для ускорения поиска наилучших гиперпараметров выполним кросс-валидацию на части выборки (20%)

In [24]:
%%time
#обучение модели
best_params_logit = grid_search_cv(model_logit, features_train['lemm_text'].sample(frac=0.2, random_state=12345).values,
                                   target_train.sample(frac=0.2, random_state=12345), parameters_logit)

F1: 0.712
Параметры лучшей модели: {'logisticregression__penalty': 'l2', 'logisticregression__solver': 'lbfgs'}

CPU times: total: 3.12 s
Wall time: 27 s


### Модель RandomForest

Для ускорения поиска наилучших гиперпараметров выполним кросс-валидацию на части выборки (20%)

In [25]:
%%time
#обучение модели
best_params_RF = grid_search_cv(model_RF, features_train['lemm_text'].sample(frac=0.2, random_state=12345).values,
                                target_train.sample(frac=0.2, random_state=12345), parameters_RF)

F1: 0.592
Параметры лучшей модели: {'randomforestclassifier__max_depth': 220}

CPU times: total: 23.1 s
Wall time: 49 s


### Модель CatBoost

Для ускорения поиска наилучших гиперпараметров выполним кросс-валидацию на части выборки (20%)

In [30]:
%%time
#обучение модели
best_params_CB = grid_search_cv(model_CB, features_train['lemm_text'].sample(frac=0.2, random_state=12345).values,
                                target_train.sample(frac=0.2, random_state=12345), parameters_CB)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

F1: 0.718
Параметры лучшей модели: {'catboostclassifier__learning_rate': 0.2}

CPU times: total: 21min 40s
Wall time: 12min 47s


### Тестирование лучшей модели с векторизацией данных TF-IDF

In [31]:
#пайплайн векторизации данных TF-IDF
pipe = make_pipeline(CountVectorizer(stop_words=stopwords), TfidfTransformer())

In [32]:
#подсчет величины TF-IDF для корпуса текстов обучающей и тестовой выборок
features_train_TF_IDF = pipe.fit_transform(features_train['lemm_text'].values)
features_test_TF_IDF = pipe.transform(features_test['lemm_text'].values)
print("Размер обучающей матрицы:", features_train_TF_IDF.shape)
print("Размер тестовой матрицы:", features_test_TF_IDF.shape)

Размер обучающей матрицы: (127433, 134106)
Размер тестовой матрицы: (31859, 134106)


In [33]:
%%time
# обучение модели
model = CatBoostClassifier(random_state=12345, learning_rate = 0.2, verbose = False)
model.fit(features_train_TF_IDF, target_train)
predictions = model.predict(features_test_TF_IDF)

f1 = f1_score(target_test, predictions)

# вывод результатов
print('F1:', round(f1, 3))
print()

F1: 0.778

CPU times: total: 1h 9min 11s
Wall time: 9min 52s


### Модель BERT

Обучение и тестирование модели BERT выполнено в Google Colab по ссылке ниже:

https://colab.research.google.com/drive/1zNSP92uxpKoXEs-SZxgFu31VfupaWmQ5?usp=sharing

Поиск наилучших гиперпараметров проведен 10% выборки:

In [34]:
print('F1 на обучающей выборке:', 0.666)
print('Параметры лучшей модели:', 'C = 0.5')

F1 на обучающей выборке: 0.666
Параметры лучшей модели: C = 0.5


Результаты обучения на тестовой выборке с взвешенными классами и силой регуляризации 0.5:

In [35]:
print('F1 на тестовой выборке:', 0.666)

F1 на тестовой выборке: 0.666


### Выводы

Кросс-валидадационный поиск лучшей модели на обучающей выборке дал следующие результаты:

In [36]:
# итоги обучения моделей
results = pd.DataFrame({'LogisticRegression': best_params_logit, 'RandomForest': best_params_RF,
                        'CatBoost': best_params_CB, 'BERT': 0.666}, index=['F1'])
display(round(results, 3))

,LogisticRegression,RandomForest,CatBoost,BERT
F1,0.712,0.592,0.718,0.666


Обучение на тестовой выборке:

In [37]:
# итоги обучения моделей
results = pd.DataFrame({'CatBoost': f1, 'BERT': 0.666}, index=['F1'])
display(round(results, 3))

,CatBoost,BERT
F1,0.778,0.666


Таким образом модель CatBoost с векторизацией данных TF-IDF показала наилучшие результаты.

## Общий вывод

В ходе работы было выполнено:

* Загружены, изучены и подготовлены данные — проведен downsampling downsampling и выполнен расчет статистической меры TF-IDF;
* Данные были разделены на обучающую и тестовую выборки;
* Обучены четыре разные модели и выбрана лучшая на валидационной выборке;
* Согласно метрике F1 лучше всего себя показала модель CatBoost с векторизацией данных TF-IDF: F1 = 0.778 на тестовой выборке.

**Таким образом, были протестированы различные модели машинного обучения и выбрана целевая перспективная модель (CatBoost) позволяющая классифицировать комментарии на позитивные и негативные.**